In [1]:
from generator import LowShotGenerator
from data_utils import *
from sklearn.model_selection import train_test_split
import pandas as pd
from classifier import Classifier

Using TensorFlow backend.


In [2]:
data_obj = get_processed_data(num_files_to_fetch_data_from=2)
le = get_label_encoder(data_obj)

X, y = get_features_and_labels(data_obj)

fetching data from file #1
fetching data from file #2


In [3]:
n_clusters = 20
λ = .5
novel_disease_name = 'Hernia'
novel_disease_label = le.transform((novel_disease_name,))[0]
n = 2
n_samples_to_generate = 20

## Train a classifier on all but one diseases

In [4]:
X_train, X_test, y_train, y_test = new_get_train_test_split_without_disease(X, y, novel_disease_name, data_obj)

cls = Classifier(n_classes=N_CLASSES - 1, n_epochs=1)
cls.fit(X_train, new_onehot_encode(y_train, [novel_disease_label]))
print("accuracy acheived: %f" % cls.evaluate(X_test, new_onehot_encode(y_test, [novel_disease_label]))[1])

Train on 10296 samples, validate on 1144 samples
Epoch 1/1
1271/1271 [==============================] - 0s 227us/step
accuracy acheived: 0.090480


## Use the trained classifier and generate new examples:

In [5]:
all_samples_features, all_samples_labels, rest_features, rest_labels = get_all_disease_samples_and_rest(X,y, novel_disease_label, data_obj)

In [6]:
X_test_with_disease, y_test_with_disease = add_disease_to_test_data(X_test, y_test, rest_features, rest_labels)

In [10]:
X_train_with_disease_samples, y_train_with_disease_samples, n_samples_features = add_n_samples_to_train_data(X_train, y_train, all_samples_features, all_samples_labels, n)

generated_features = n_samples_features
# generated_features = LowShotGenerator.get_generated_features(cls,
#                                                              novel_disease_label,
#                                                              n_samples_features,
#                                                              n_clusters,
#                                                              λ,
#                                                              n_samples_to_generate)

## Train a classifier on real n_samples of one disease

In [11]:
cls = Classifier(n_classes=N_CLASSES, n_epochs=1)
cls.fit(X_train_with_disease_samples, new_onehot_encode(y_train_with_disease_samples))
print("accuracy acheived: %f" % cls.evaluate(X_test_with_disease, new_onehot_encode(y_test_with_disease))[1])

Train on 10298 samples, validate on 1145 samples
Epoch 1/1
1271/1271 [==============================] - 1s 987us/step
accuracy acheived: 0.166011


## Train a classifier on real n_samples + generated samples

In [12]:
generated_data_label = novel_disease_label
X_train_with_generated_data, y_train_with_generated_data = add_generated_data_to_train_data(X_train_with_disease_samples, y_train_with_disease_samples, generated_features, generated_data_label)

cls = Classifier(n_classes=N_CLASSES, n_epochs=1)
cls.fit(X_train_with_generated_data, new_onehot_encode(y_train_with_generated_data))
print("accuracy acheived: %f" % cls.evaluate(X_test_with_disease, new_onehot_encode(y_test_with_disease))[1])

Train on 10301 samples, validate on 1145 samples
Epoch 1/1
1271/1271 [==============================] - 0s 196us/step
accuracy acheived: 0.105429
